# Ibsens xmlStrip - *«for å fjerne diverse buskvekster»*

In [1]:
import time
print("Dette programmet tar XML-filer med Ibsenstruktur og gjør dem om til rentekstfiler. Trykk STOPP nå om du ikke vil kjøre.")
print("Har du husket å sette riktige parametre i cellen under?")
time.sleep(5)

Dette programmet tar XML-filer med Ibsenstruktur og gjør dem om til rentekstfiler. Trykk STOPP nå om du ikke vil kjøre.
Har du husket å sette riktige parametre i cellen under?


# Parametre
Under har du parametre for konversjonen.

## his:hisdel
tag_hisdel kontrollerer følgende undertagger: *overwritten*, *overstrike*. Om tag_hisdel = 2 erstattes alle hisdel-taggene med markdown (~), om tag_hisdel = 1 slettes alle helt, og om tag_hisdel er 0 kontrolleres hver type av sin egen verdi. Dvs at GRUPPEN tar prioritet over individuell tagg.

BUG: 2 fungerer ikke... helt. Vet ikke hvorfor. Den finner his:hisdel-taggen(e), men erstatningen inntreffer ikke. Må sannsynligvis bruke en annen metode for å erstatte. Det kan også være fordi disse hisdel-taggene kan være nøstet inni hverandre, kanskje.

In [2]:
# 0 gjør ingenting, 1 sletter taggen med innholdet HELT, 2 beholder innholdet med markdown (~~)
# Gruppekontroll
tag_hisdel = 1

# Individuell kontroll - brukes kun dersom gruppekontroll er 0
# DISSE FUNGERER IKKE ENDA - BRUK GRUPPEKONTROLL 1 ELLER 2, 3 FOR Å DEAKTIVERE. IKKE 0
tag_overwritten = 1
tag_overstrike = 1
tag_diagonaloverstrike = 1

# Program
Ikke rør noe under om du ikke vet hva du gjør (med mindre du vet hvordan du spammer undo til det fungerer igjen ;). ).

In [3]:
from bs4 import BeautifulSoup # Hent BeautifulSoup-modulen (https://www.crummy.com/software/BeautifulSoup/)
import re # Requests. Brukes i noen regex-funksjoner, gitt
import os # Filsystem for lasting og lagring
import codecs # Dekoder for ASCII
import csv # CSV-leser for ASCII

In [17]:
#Etabler liste over koder og ascii

file = open('translation.csv') # Egendefinert fil fra DTD. &nbsp, &shy og &tu20 er manuelt oversatt til [mellomrom], [-] og [mellomrom].
# Andre diverse entiteter som manglet er også oversatt.
csvreader = csv.reader(file) # Importer  filen over
asciis = [] # Array for unicode-greier (eks. \u0023)
codes = [] # Array for ting som &nbsp - "html"-koder
for row in csvreader: # For hver rad
    htmlcode = row[0] # Html-tagger (de som er synlige i teksten)
    unicode = row[1] # Unicode (eg \u0023)
    ASCII = codecs.decode(unicode, 'unicode_escape') # Gjør om til ASCII
    asciis.append(ASCII) # Legg till i array
    codes.append(htmlcode)
    print(htmlcode,"=>",ASCII)

&half; => ½
&#x0254; => Ãâ
&frac12; => ½
&frac14; => ¼
&frac34; => ¾
&frac18; => ⅛
&frac38; => ⅜
&frac58; => ⅝
&frac78; => ⅞
&sup1; => ¹
&sup2; => ²
&sup3; => ³
&plus; => +
&plusmn; => ±
&equals; => =
&divide; => ÷
&times; => ×
&curren; => ¤
&num; => #
&percnt; => %
&ast; => ∗
&commat; => @
&bsol; => \
&lcub; => {
&horbar; => ―
&verbar; => |
&rcub; => }
&micro; => µ
&ohm; => Ω
&deg; => °
&ordm; => º
&ordf; => ª
&sect; => §
&para; => ¶
&middot; => ·
&larr; => ←
&rarr; => →
&uarr; => ↑
&darr; => ↓
&copy; => ©
&reg; => ®
&trade; => ™
&brvbar; => ¦
&not; => ¬
&sung; => ♩
&excl; => !
&iexcl; => ¡
&quot; => "
&apos; => '
&lpar; => (
&rpar; => )
&comma; => ,
&lowbar; => _
&hyphen; => -
&period; => .
&sol; => /
&colon; => :
&semi; => ;
&quest; => ?
&iquest; => ¿
&laquo; => «
&raquo; => »
&lsquo; => ‘
&rsquo; => ’
&ldquo; => “
&rdquo; => ”
&nbsp; =>  
&shy; => -
&acute; => ´
&breve; => ˘
&caron; => ˇ
&cedil; => ¸
&circ; => ^
&dblac; => ˝
&die; => ¨
&dot; => ˙
&grave; => `
&macr; => ¯
&ogon; 

In [5]:
#Hvilke tagger skal utraderes MED innhold?
tags_to_erase = ["handnote","teiheader","front","lb","figure","pb","note","anchor","back"]
if tag_hisdel == 1:
    tags_to_erase.append("his:hisdel")
elif tag_hisdel == 2:
    print("Alle <his:hisdel>-tagger blir erstattet med markdown: { ~~ [innhold] ~~ }")
    print("Obs, dette fungerer ikke enda. :/")
    # for tag in content.findAll('his:hisdel'):
        # tag.replaceWith("~~ %s ~~" % a.string)
elif tag_hisdel == 0:
    if tag_overwritten == 1:
        tags_to_erase.append("overwritten") 
        # content.findAll("his:hisdel", {"rend" : "overwritten"})
    elif tag_overwritten == 2:
        print("TBI: markdown around overwrittens")
    if tag_overstrike == 1:
        tags_to_erase.append("overstrike") 
        # content.findAll("his:hisdel", {"rend" : "overstrike"})
    elif tag_overstrike == 2:
        print("TBI: markdown around overstrikes")
    if tag_diagonalOverstrike == 1:
        tags_to_erase.append("diagonalOverstrike")
        # content.findAll("his:hisdel", {"rend" : "diagonalOverstrike"})
    elif tag_overstrike == 2:
        print("TBI: markdown around diagonal overstrikes")

In [6]:
# GLOB GLOB GOBGALAB
import glob
paths = []
for path in glob.glob('xml_src/**/*.xml', recursive=True):
    paths.append(path)
#paths # Liste over stier til filer i SRC
errors = []
outputs = []
for path in paths:
    path = path.replace('xml_src','xml_output')
    outputs.append(path)
#outputs # Liste over stier til nye filer

['xml_output\\brev\\B1844-1871ht.xml',
 'xml_output\\brev\\B1871-1879ht.xml',
 'xml_output\\brev\\B1880-1889ht.xml',
 'xml_output\\brev\\B1890-1905ht.xml',
 'xml_output\\dikt\\BlaDigt8191.xml',
 'xml_output\\dikt\\Di01.xml',
 'xml_output\\dikt\\Di41110a.xml',
 'xml_output\\dikt\\Di41113h.xml',
 'xml_output\\dikt\\Di42622.xml',
 'xml_output\\dikt\\Di71.xml',
 'xml_output\\dikt\\Di75.xml',
 'xml_output\\dikt\\Di79.xml',
 'xml_output\\dikt\\Di81928.xml',
 'xml_output\\dikt\\Di82.xml',
 'xml_output\\dikt\\Di86.xml',
 'xml_output\\dikt\\Di91.xml',
 'xml_output\\dikt\\Di96.xml',
 'xml_output\\dikt\\DiFU.xml',
 'xml_output\\dikt\\Diktht.xml',
 'xml_output\\drama\\BE\\BE428691_KBK.xml',
 'xml_output\\drama\\BE\\BE428691_KBK_gml.xml',
 'xml_output\\drama\\BE\\BE428691_KBK_original.xml',
 'xml_output\\drama\\BE\\BEht.xml',
 'xml_output\\drama\\BE\\BEht_noter.xml',
 'xml_output\\drama\\BE\\BEht_original.xml',
 'xml_output\\drama\\BE\\BEht_ut.xml',
 'xml_output\\drama\\Br\\Br03.xml',
 'xml_output\

In [7]:
# ACTION
i = 0
x = 0
y = 0
for thing in paths: # For hver sti (med endestasjon fil) i listen over stier
    # Filhåndtering I - Les og last xml-filer inn i minne
    with open(thing, "r", encoding="utf-8") as file: # Åpne stien, les, med encoding UTF8
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # Combine the lines in the list into a string
        content = "".join(content) # Variabel content får innholdet
    
    # Filhåndtering II - Lag ny sti
    frags = outputs[i].split('\\') # Splitt outputstrengen for sti
    if "xml" in frags[2]: # Hvis filen er 2 ned, altså vi finner xml-filen her (eks. xml_src\dikt\~.xml)
        dirName = frags[0]+"\\"+frags[1] # Dirname blir 2 dyp (eks. xml_output\dikt\)
    else: # OBS!! OBS!! Hvis den IKKE er 2 ned ANTAR vi at den er 3 ned - som i DRAMA!!!
        dirName = frags[0]+"\\"+frags[1]+"\\"+frags[2] # Dirname blir 3 ned (xml_output\drama\XYZ)
        archDir = frags[0]+"\\"+frags[1] # FAILSAFE for DRAMA: Drama er ett hakk lengre ned i hierarkiet. Skap mappe for å ta drama
        #OBS - koden her ANTAR at senteret ALLTID følger samme praksis. Koden må endres om denne endres.
        if not os.path.exists(archDir): # Hvis subfolder ikke finnes (eks.\drama\BE\)
            os.mkdir(archDir) # Skap den
            #print("Created",archDir,"for deep path")
    outputs[i] = outputs[i][:-4] # Strip 4 tegn fra slutten av hver streng for å få bort .xml (4. 3X 2M 1L)
    # Create target Directory if don't exist
    if not os.path.exists(dirName):
        os.mkdir(dirName)
        #print("Created",dirName,"from full path")
    name = outputs[i]+"_SANERT.txt" # Outputfil får navnet X_SANERT.txt
    
    # XML til TXT - Det er her vi skal sette inn X FOR EACH for å redigere XML-filene.
    
    content = BeautifulSoup(content, from_encoding="UTF-8") # Content er content som bs4-objekt...
    
    # Hent strukturer vi skal beholde
    try:
        titlestatement = content.find("titlestmt") # Finn title statement
        titlestatement.extract() # Hent (og fjern) title statement-delen
    except:
        print("Ingen title statement.")

    try:
        filedescription = content.find("filedesc") # Finn filbeskrivelsen
        filedescription.extract() # Hent (og fjern) filbeskrivelsen
    except:
        print("Ingen filbeskrivelse.")
    # Slett tagger vi ikke skal beholde
    if tag_hisdel == 2:
        z = 0
        try:
            for tag in content.findAll('his:hisdel'):
                tag.replaceWith(" ~~ %s ~~ " % a.string)
                z += 1
            if z > 0:
                print(">Erstattet",z,"hisdel-tagger med markdown-tagger.")
            else:
                print(">his:hisdel-tagger er ikke 0, men 0 ble fjernet...? Hva.")
        except:
            print("Ingen hisdel-tagger å endre.")
    try:
        for tag in tags_to_erase:
            to_remove = content.find_all(tag) # Finn alle instanser av taggen
            for remove in to_remove: # For hver av disse
                remove.decompose() # Fjern taggen og innholdet helt
    except:
        print("Oi, det var det en feil med",tag,remove)

    # Taggen his:hisdel rend="overstrike" - skal denne slettes? nei! Legg inn blanktegn eller annen formatering som markerer.
    
    content = str(content) # CONTENT ER NÅ STRING - STRINGMETODER FUNGERER, BS4-METODER FUNGERER IKKE
    
    z = 0 # Tell fra 0 i z
    for code in codes: # For hver kode
        if code in content: # Hvis koden finnes i teksten
            content = content.replace(code,asciis[z]) # Erstatt koden med ascii
            #print('Erstattet',code,'med',asciis[z]) # Print bekreftelse
        z+=1 # Neste kode/ascii etter å ha sjekket hver kode
    
    content = content.replace("\n"," ") # Slett newline og erstatt med mellomrom
    # "Ordinær" RegEx-vask - den er ikke lenger ordinær...
    content = content.replace(".</l>",".") # Fjern slutten på L og erstatt med et punktum der det er punktum-.. vent.
    # Hvorfor gjorde jeg det over, igjen...?
    content = re.sub('<[^<]+>', "", content) # Slett alle resterende tagger UTEN å røre innholdet
    content = re.sub("'“'", "", content) # Korriger enkelte instanser av '“'
    content = re.sub("'\"'", "", content) # Korriger enkelte instanser av '"'...
    # De to over kan kanskje fjernes nå som jeg har fikset litt i translation.csv... kanskje.
    content = re.sub("&dash;","–",content) # Korriger &dash; fordi den ikke vil korrigeres gjennom translation.csv??
    content = re.sub("&", " & ", content) # Her finner vi alle ampersander og legger mellomrom rundt. Disse slettes etterpå.
    content = re.sub('\s+',' ', content) # Korriger alle instanser av multimellomrom til enkeltmellomrom
    
    content = re.sub("& &", "&", content) # Korriger dobbeltampersander etter mellomrom er fjernet.
    content = re.sub(". &", ".", content) # Starter da ikke en setning med ampersand-tegn, vel...?
    
    # Filhåndtering III - Skriv til disk
    try: # Prøv å skrive til disk
        f = open(name, "w", encoding="utf-8") # Ikke glem encoding="utf-8", sånn som jeg gjorde.
        f.write(content)
        f.close()
        print("Opprettet",name,"i",dirName)
        x+=1
    except: # Hvis vi møter på feil, rapporter kilden
        print("FILFEIL!",thing)
        f = open(name+"_ERROR", "w", encoding="utf-8")
        f.write("å neeeiii")
        f.close()
        errors.append(name)
        y+=1
    i+=1
#print(content)
print("\n",x,"files parsed to text.",y,"files failing to parse.")
if y > 0:
    face = ":("
else:
    face = ":)"

C:\Users\lokes\Anaconda3\lib\site-packages\bs4\__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


Opprettet xml_output\brev\B1844-1871ht_SANERT.txt i xml_output\brev
Opprettet xml_output\brev\B1871-1879ht_SANERT.txt i xml_output\brev
Opprettet xml_output\brev\B1880-1889ht_SANERT.txt i xml_output\brev
Opprettet xml_output\brev\B1890-1905ht_SANERT.txt i xml_output\brev
Opprettet xml_output\dikt\BlaDigt8191_SANERT.txt i xml_output\dikt
Opprettet xml_output\dikt\Di01_SANERT.txt i xml_output\dikt
Opprettet xml_output\dikt\Di41110a_SANERT.txt i xml_output\dikt
Opprettet xml_output\dikt\Di41113h_SANERT.txt i xml_output\dikt
Opprettet xml_output\dikt\Di42622_SANERT.txt i xml_output\dikt
Opprettet xml_output\dikt\Di71_SANERT.txt i xml_output\dikt
Opprettet xml_output\dikt\Di75_SANERT.txt i xml_output\dikt
Opprettet xml_output\dikt\Di79_SANERT.txt i xml_output\dikt
Opprettet xml_output\dikt\Di81928_SANERT.txt i xml_output\dikt
Opprettet xml_output\dikt\Di82_SANERT.txt i xml_output\dikt
Opprettet xml_output\dikt\Di86_SANERT.txt i xml_output\dikt
Opprettet xml_output\dikt\Di91_SANERT.txt i xml

Opprettet xml_output\drama\F2\F28950_SANERT.txt i xml_output\drama\F2
Opprettet xml_output\drama\F2\F291_SANERT.txt i xml_output\drama\F2
Opprettet xml_output\drama\F2\F298_SANERT.txt i xml_output\drama\F2
Opprettet xml_output\drama\F2\F2FU_SANERT.txt i xml_output\drama\F2
Opprettet xml_output\drama\F2\F2ht_SANERT.txt i xml_output\drama\F2
Opprettet xml_output\drama\F2\F2ht_noter_SANERT.txt i xml_output\drama\F2
Opprettet xml_output\drama\FH\FH02_SANERT.txt i xml_output\drama\FH
Opprettet xml_output\drama\FH\FH166_SANERT.txt i xml_output\drama\FH
Opprettet xml_output\drama\FH\FH41116a_1-2_4_3_SANERT.txt i xml_output\drama\FH
Opprettet xml_output\drama\FH\FH41116a_11r_SANERT.txt i xml_output\drama\FH
Opprettet xml_output\drama\FH\FH41116a_11v_SANERT.txt i xml_output\drama\FH
Opprettet xml_output\drama\FH\FH41116a_3_7-8_SANERT.txt i xml_output\drama\FH
Opprettet xml_output\drama\FH\FH41116a_5r_6v_SANERT.txt i xml_output\drama\FH
Opprettet xml_output\drama\FH\FH41116a_5v_SANERT.txt i xml_

Opprettet xml_output\drama\Hv\Hv41291c_SANERT.txt i xml_output\drama\Hv
Opprettet xml_output\drama\Hv\Hv41291c_nesten_orig_SANERT.txt i xml_output\drama\Hv
Opprettet xml_output\drama\Hv\Hvht_SANERT.txt i xml_output\drama\Hv
Opprettet xml_output\drama\Hv\Hvht_noter_SANERT.txt i xml_output\drama\Hv
Opprettet xml_output\drama\Hv\Hvht_orig_SANERT.txt i xml_output\drama\Hv
Opprettet xml_output\drama\JG\JG41118a_SANERT.txt i xml_output\drama\JG
Opprettet xml_output\drama\JG\JG41118b_SANERT.txt i xml_output\drama\JG
Opprettet xml_output\drama\JG\JG42590_SANERT.txt i xml_output\drama\JG
Opprettet xml_output\drama\JG\JG4262IV3_SANERT.txt i xml_output\drama\JG
Opprettet xml_output\drama\JG\JG96_SANERT.txt i xml_output\drama\JG
Opprettet xml_output\drama\JG\JG96a_SANERT.txt i xml_output\drama\JG
Opprettet xml_output\drama\JG\JGFU_SANERT.txt i xml_output\drama\JG
Opprettet xml_output\drama\JG\JGht_SANERT.txt i xml_output\drama\JG
Opprettet xml_output\drama\JG\JGht_noter_SANERT.txt i xml_output\dra

Opprettet xml_output\drama\NV\NV41119b_SANERT.txt i xml_output\drama\NV
Opprettet xml_output\drama\NV\NV4262IV4_SANERT.txt i xml_output\drama\NV
Opprettet xml_output\drama\NV\NV99_SANERT.txt i xml_output\drama\NV
Opprettet xml_output\drama\NV\NV99a_SANERT.txt i xml_output\drama\NV
Opprettet xml_output\drama\NV\NVFU_SANERT.txt i xml_output\drama\NV
Opprettet xml_output\drama\NV\NVht_SANERT.txt i xml_output\drama\NV
Opprettet xml_output\drama\NV\NVht_noter_SANERT.txt i xml_output\drama\NV
Opprettet xml_output\drama\NV\NVLondon_SANERT.txt i xml_output\drama\NV
Opprettet xml_output\drama\O1\O1344_Alfhild_SANERT.txt i xml_output\drama\O1
Opprettet xml_output\drama\O1\O1344_AndenSvend_SANERT.txt i xml_output\drama\O1
Opprettet xml_output\drama\O1\O1344_Bryllupsgjaester_SANERT.txt i xml_output\drama\O1
Opprettet xml_output\drama\O1\O1344_FoersteSvend_SANERT.txt i xml_output\drama\O1
Opprettet xml_output\drama\O1\O1344_FruKirsten_SANERT.txt i xml_output\drama\O1
Opprettet xml_output\drama\O1\O

Opprettet xml_output\drama\UF\UF41109a_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UF41109b_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UF4262_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UF69_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UF69a_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UF74_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UF80_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UF81924_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UF81937a_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UF83_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UF95_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UFFU_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UFht_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\drama\UF\UFht_noter_SANERT.txt i xml_output\drama\UF
Opprettet xml_output\dra

In [8]:
print("Gjorde om",i+1,"dokumenter til rentekst fra",len(paths)+1,"XML-filer. Vi fikk",y,"encoding-feil.",face)
if errors:
    errors

Gjorde om 494 dokumenter til rentekst fra 494 XML-filer. Vi fikk 0 encoding-feil. :)


~~UnicodeEncodeError: 'charmap' codec can't encode character '\u203b' in position 1180: character maps to undefined. Denne karakteren her: ※ (Fixed)~~

~~Ok, fra 3 feil til 1 etter å ha konvertert "&ergo;" til "ɔ"!~~
Header
FIX: Glemte å manuelt sette output til encoding="utf-8"... men nå skriver alle filer!

TODO: Mange korreksjoner og sånne ting som gjemmer seg blant teksten i tags.

TODO: De variablene under må dekonstrueres og deler av dem må stappes inn igjen.

In [10]:
yo = str(filedescription)+str(titlestatement)

yo

'<filedesc>\n\n<editionstmt><edition xml:id="P1841-1905ht-0212"></edition></editionstmt>\n<publicationstmt>\n<distributor xml:id="HISe">Universitetet i Oslo</distributor>\n<date when="2012" xml:id="HISe-date"></date>\n<pubplace>Oslo</pubplace>\n</publicationstmt>\n<sourcedesc>\n<p>Denne fila bygger på P1841-1905ht_bok.xml (P1841-1905ht-0810)\n<list>\nGrunntekstfiler\n<item>P1841-59gt.xml</item>\n<item>P1860-69gt.xml</item>\n<item>P1870-79gt.xml</item>\n<item>P1880-89gt.xml</item>\n<item>P1890-1900gt.xml</item>\n<item>Nye_gt_sakprosa.xml</item>\n<item>Flere_nye_sakprosatekster.xml</item>\n</list></p>\n<p>\n<bibl xml:id="P1841Drom">\n<idno type="owner">NB depot</idno>\n<idno>EOAA00034122</idno>\n<author>Henrik Ibsen</author>\n<title type="main">Fra Bygderne</title>\n<title type="sub">Ny række. II. Henrik Ibsen. &amp;dash; Maalstrævere. &amp;dash; &amp;sol; Hartvig Lassen.</title>\n<title type="occasion">[Gjenfortalt skolestil; en drøm]</title>\n<title>Fædrelandet (onsdag, nr. 40, 11. årg

# Ferdig!
De rene tekstfilene skal nå finnes under .../xml_output/*. Med mindre du fikk feil i scriptet over. Eller om noe annet gikk galt.

___

# Testing
Ting under denne overskriften er bare å se bort fra med mindre du skal teste nye greier eller andre løsninger.

In [11]:
thing = paths[24]
with open(thing, "r", encoding="utf-8") as file: # Åpne stien, les, med encoding UTF8
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content) # Variabel content får innholdet
content = BeautifulSoup(content, from_encoding="UTF-8") # Content er content som bs4-objekt...

In [12]:
try:
    for tag in content.findAll('his:hisdel'):
        tag.replaceWith(" ~~ %s ~~ " % a.string)
        z += 1
    if z > 0:
        print(">Erstattet",z,"hisdel-tagger med markdown-tagger.")
    else:
        print(">his:hisdel-tagger er ikke 0, men 0 ble fjernet...? Hva.")
except:
    print("Ingen hisdel-tagger å endre.")
content

Ingen hisdel-tagger å endre.


<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE TEI>
<html><body><p>
%HIS_entiteter;
]&gt;
<?xml-model href="http://www.edd.uio.no/ibsen/schema/tei_his.rnc" type="application/relax-ng-compact-syntax"?>
<tei xmlns="http://www.tei-c.org/ns/1.0" xmlns:his="http://www.example.org/ns/HIS">
<teiheader xml:lang="nob">
<filedesc>
<titlestmt>
<title level="a" type="main">[Episk Brand]</title>
<title level="s" type="main">Henrik Ibsens skrifter 2007</title>
<title level="s" type="sub">Hovedtekst, tekstgrunnlag: KBK NKS 2869, 4°, 1</title>
<title level="s" type="copyright">© Henrik Ibsens skrifter</title>
<author>Henrik Ibsen</author>
<funder>&amp;funder;</funder>
<editor>&amp;editor;</editor>
<respstmt type="editing"><resp>Edering</resp><name>Christian Janss (ansv.), Hilde Bøe, Ellen Nessheim Wiger</name></respstmt>
<respstmt type="encoding"><resp>Kollasjonering og koding</resp><name>Hilde Bøe, Henninge Margrethe Solberg, Ellen Nessheim Wiger</name></respstmt>
<respstmt type="commenting"><resp>K

In [13]:
try:
    to_remove = content.findAll('his:hisdel') # Finn alle instanser av taggen
    for remove in to_remove: # For hver av disse
        remove.decompose() # Fjern taggen og innholdet helt
except:
    print("Oi, det var det en feil med",tag,remove)
content

<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE TEI>
<html><body><p>
%HIS_entiteter;
]&gt;
<?xml-model href="http://www.edd.uio.no/ibsen/schema/tei_his.rnc" type="application/relax-ng-compact-syntax"?>
<tei xmlns="http://www.tei-c.org/ns/1.0" xmlns:his="http://www.example.org/ns/HIS">
<teiheader xml:lang="nob">
<filedesc>
<titlestmt>
<title level="a" type="main">[Episk Brand]</title>
<title level="s" type="main">Henrik Ibsens skrifter 2007</title>
<title level="s" type="sub">Hovedtekst, tekstgrunnlag: KBK NKS 2869, 4°, 1</title>
<title level="s" type="copyright">© Henrik Ibsens skrifter</title>
<author>Henrik Ibsen</author>
<funder>&amp;funder;</funder>
<editor>&amp;editor;</editor>
<respstmt type="editing"><resp>Edering</resp><name>Christian Janss (ansv.), Hilde Bøe, Ellen Nessheim Wiger</name></respstmt>
<respstmt type="encoding"><resp>Kollasjonering og koding</resp><name>Hilde Bøe, Henninge Margrethe Solberg, Ellen Nessheim Wiger</name></respstmt>
<respstmt type="commenting"><resp>K

In [14]:
content = "aa aA a         HADbhwiibdwa zofnofa \n riaeraj  ra ra  ar a a a "
content

'aa aA a         HADbhwiibdwa zofnofa \n riaeraj  ra ra  ar a a a '